In [26]:
# !pip install langchain_community

In [27]:
from langchain_community.llms import Ollama

In [28]:
llm = Ollama(model="llama2")
llm.invoke("The first man on the moon was ...")

'\nThe first man on the moon was Neil Armstrong. He stepped foot on the lunar surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first person to walk on the moon.'

In [29]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
ISS Catering Services De Meern, Vishandel Sier AMSTELVEEN, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

'\nSure! Here are the categories for each of the expenses you provided:\n\n* Spotify AB by Adyen - Entertainment\n* Beta Boulders Ams Amsterdam Nld - Sports\n* ISS Catering Services De Meern - Food and Beverage\n* Vishandel Sier AMSTELVEEN - Retail\n* Ministerie van Justitie en Veiligheid - Government\n* Etos AMSTERDAM NLD - Retail\n* Bistro Bar Amsterdam - Food and Beverage'

### Read transaction data

In [30]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (Rupee)
0,12/30/2023,Raj's Dhaba,Expense,500
1,12/30/2023,Airtel Bill Payment,Expense,300
2,12/30/2023,House Rent,Expense,15000
3,12/30/2023,Sweta's Boutique Purchase,Expense,1200
4,12/29/2023,Freelance Web Design Project,Income,8000


In [31]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

19

In [32]:
unique_transactions[1:10]

array(['Airtel Bill Payment', 'House Rent', "Sweta's Boutique Purchase",
       'Freelance Web Design Project', 'Zomato Food Order',
       'Grocery from D-Mart', 'Tuition Fees Received',
       'Diwali Donation to NGO', 'Salary'], dtype=object)

### Categorise bank transactions with Llama2

In [33]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 19]

In [34]:
# Output validation
from pydantic import BaseModel, field_validator
from typing import List

# Validate response format - check if it actually contains hyphen ("-")
class ResponseChecks(BaseModel):
    data: List[str]

    @field_validator("data")
    def check(cls, value):
        for item in value:
            if len(item) > 0:
                assert "-" in item, "String does not contain hyphen."

# Test validation
ResponseChecks(data = ['Hello - World', 'Hello - there!'])

ResponseChecks(data=None)

In [35]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    # Keep only the lines in between blank lines (removing the explaination lines at the beginning and end of the response)
    blank_indexes = [index for index in range(len(response)) if response[index] == '']
    if len(blank_indexes) == 1:
        response = response[(blank_indexes[0] + 1):]
    else:
        response = response[(blank_indexes[0] + 1) : blank_indexes[1]]

    # Print response and validate if it is in the correct format
    print(response)
    ResponseChecks(data = response)
    
    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [41]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Vishandel Sier AMSTELVEEN, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['Sure! Here are the appropriate categories for each of the expenses you listed:']


ValidationError: 1 validation error for ResponseChecks
data
  Assertion failed, String does not contain hyphen. [type=assertion_error, input_value=['Sure! Here are the appr...e expenses you listed:'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.6/v/assertion_error

In [ ]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()
max_tries = 7

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    # Try and validate output, if it fails, try again for max_tries=7 times
    for j in range(1, max_tries):
        try:
            categories_df = categorize_transactions(transaction_names, llm)
            categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)
            
        except:
            if j < max_tries:
                continue
            else:
                raise Exception(f"Cannot categorise transactions indexes {i} to {i+1}.")
        break

TypeError: sequence item 19: expected str instance, float found

In [ ]:
categories_df_all.head()

""


In [ ]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [ ]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

In [ ]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [ ]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

In [ ]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

In [ ]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)